In [12]:
import pandas as pd
import numpy as np
import requests
import statsmodels.api as sm
from pandas.tseries.offsets import BDay


In [13]:
import matplotlib.pyplot as plt

abnormal_returns = pd.read_csv('abnormal_returns.csv', sep=';')

# Load the CSV file, replacing commas with dots and ensuring all data is treated as floats
prices = pd.read_csv('Extrarow_with_empty_rows_update.csv', sep=';', dtype=str)

# Replace NaNs with zeros
prices = prices.fillna(0)

for i in range(1, len(prices), 2):
    prices.at[i, 'Code'] = 'Rm'

# Convert the 'T-1Y' and 'T+1Y' columns to datetime format
prices['T-1Y'] = pd.to_datetime(prices['T-1Y'], format='%d/%m/%Y', errors='coerce')
prices['T+1Y'] = pd.to_datetime(prices['T+1Y'], format='%d/%m/%Y', errors='coerce')

# Subtract 5 days from the dates in the 'T-1Y' and 'T+1Y' columns
prices['T-1Y'] = prices['T-1Y'] - pd.Timedelta(days=5)
prices['T+1Y'] = prices['T+1Y'] - pd.Timedelta(days=5)

print(prices.head())

# Drop irrelevant columns
prices = prices.drop('SPLIT', axis=1)

# Replace commas with dots and convert to float where possible
prices = prices.replace({',': '.'}, regex=True).apply(pd.to_numeric, errors='ignore')

# Convert 'Date' to datetime format, handling errors
prices['date'] = pd.to_datetime(prices['Date'], format='%d/%m/%Y', errors='coerce')

# Create a DataFrame to store the abnormal returns
abnormal_returns = pd.DataFrame()


def plot_abnormal_returns(ticker, abnormal_returns_df):
    """
    Plot the abnormal returns for a given ticker over time.
    
    Parameters:
    ticker (str): The ticker symbol of the firm whose abnormal returns to plot.
    abnormal_returns_df (pd.DataFrame): The DataFrame containing abnormal returns.
    """
    # Check if the ticker exists in the DataFrame
    if ticker not in abnormal_returns_df.columns:
        print(f"Ticker {ticker} not found in the abnormal returns data.")
        return

    # Extract the abnormal returns for the given ticker
    abnormal_data = abnormal_returns_df[ticker].dropna()

    # Create the plot
    plt.figure(figsize=(10, 6))
    plt.plot(abnormal_data.index, abnormal_data.values, label=f'Abnormal Returns - {ticker}', marker='o')
    
    # Add labels and title
    plt.xlabel('Date')
    plt.ylabel('Abnormal Return')
    plt.title(f'Abnormal Returns Over Time for {ticker}')
    plt.grid(True)
    plt.legend()

    # Display the plot
    plt.show()

# Example usage:
# Assuming `abnormal_returns` is your DataFrame containing abnormal returns,
# and you want to plot the abnormal returns for the ticker "SKIL":
plot_abnormal_returns("FWFW", abnormal_returns)


   Code                        Name          ISIN        Date Factor  \
0  SKIL      Skillsoft Corp Class A  US83066P3091  10/02/2023     20   
1    Rm                           0             0           0      0   
2  VISL  Vislink Technologies, Inc.  US92836Y4098  05/01/2023     20   
3    Rm                           0             0           0      0   
4  MARK       Remark Holdings, Inc.  US75955K3005  22/12/2022     10   

        T-1Y       T+1Y        T-250        T-249       T-248  ...  \
0 2022-02-05 2024-02-05   -1,7241418    2,0733595    -2,34375  ...   
1        NaT        NaT  -0,38406253     1,576674  0,08811951  ...   
2 2021-12-31 2023-12-31   0,90090036   -4,4642744    4,672885  ...   
3        NaT        NaT  -0,40501952  -0,14410019   0,9160042  ...   
4 2021-12-17 2023-12-17   -3,2843113   -3,7404895   3,9069057  ...   

         T+241      T+242       T+243       T+244        T+245        T+246  \
0   0,33534765   2,272725  -7,2549047  -1,1980236     6,562042   -1

/var/folders/3k/zwv52hz93g96b2yjn3w9fywm0000gn/T/ipykernel_79754/2805036603.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  prices = prices.replace({',': '.'}, regex=True).apply(pd.to_numeric, errors='ignore')


In [14]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_abnormal_returns_with_split(ticker, abnormal_returns_df, prices_df):
    """
    Plot the abnormal returns for a given ticker over time with enhanced visualization,
    and annotate and mark the stock split date.
    
    Parameters:
    ticker (str): The ticker symbol of the firm whose abnormal returns to plot.
    abnormal_returns_df (pd.DataFrame): The DataFrame containing abnormal returns.
    prices_df (pd.DataFrame): The DataFrame containing stock split dates.
    """
    # Check if the ticker exists in the DataFrame
    if ticker not in abnormal_returns_df.columns:
        print(f"Ticker {ticker} not found in the abnormal returns data.")
        return

    # Extract the abnormal returns for the given ticker
    abnormal_data = abnormal_returns_df[ticker].dropna()

    # Extract the split date for the given ticker from the prices DataFrame
    split_date = prices_df.loc[prices_df['Code'] == ticker, 'date'].values
    if len(split_date) > 0:
        split_date = pd.to_datetime(split_date[0])
    else:
        print(f"No split date found for ticker {ticker}.")
        split_date = None

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 6))

    # Plot the abnormal returns
    ax.plot(abnormal_data.index, abnormal_data.values, label=f'Abnormal Returns - {ticker}', marker='o', color='b', lw=2)

    # Set custom x-axis ticks every 40 points to avoid crowding
    ax.set_xticks(abnormal_data.index[::40])

    # Format the x-axis ticks to display dates (if available) or sequential numbers
    if isinstance(abnormal_data.index, pd.DatetimeIndex):
        ax.set_xticklabels(abnormal_data.index[::40].strftime('%b %d, %Y'), rotation=45, ha='right')
    else:
        ax.set_xticklabels(abnormal_data.index[::40], rotation=45, ha='right')

    # Highlight max and min abnormal returns
    max_return = abnormal_data.max()
    min_return = abnormal_data.min()
    max_date = abnormal_data.idxmax()
    min_date = abnormal_data.idxmin()

    # Annotate the max point
    ax.annotate(f'Max: {max_return:.2f}', xy=(max_date, max_return), xytext=(max_date, max_return + 0.5),
                arrowprops=dict(facecolor='green', shrink=0.05), fontsize=10, color='green')

    # Annotate the min point
    ax.annotate(f'Min: {min_return:.2f}', xy=(min_date, min_return), xytext=(min_date, min_return - 0.5),
                arrowprops=dict(facecolor='red', shrink=0.05), fontsize=10, color='red')

    # Annotate and mark the split date with an arrow and a distinct marker
    if split_date and split_date in abnormal_data.index:
        split_return = abnormal_data.loc[split_date]

        # Annotate the split date
        ax.annotate('Split Date', xy=(split_date, split_return), xytext=(split_date, split_return + 1),
                    arrowprops=dict(facecolor='orange', shrink=0.05), fontsize=10, color='orange')

        # Add a marker at the exact split point
        ax.plot(split_date, split_return, marker='x', markersize=12, color='orange', label='Split Date Marker')

    # Add labels and title
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Abnormal Return', fontsize=12)
    ax.set_title(f'Abnormal Returns Over Time for {ticker} (Split Date Marked)', fontsize=14)

    # Add grid, legend, and improve layout
    ax.grid(True)
    ax.legend()
    plt.tight_layout()

    # Display the plot
    plt.show()

# Example usage:
# Assuming `abnormal_returns` is your DataFrame containing abnormal returns,
# and `prices` is your DataFrame containing the stock split dates for each ticker:
plot_abnormal_returns_with_split("SKIL", abnormal_returns, prices)


Ticker SKIL not found in the abnormal returns data.


In [15]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_abnormal_returns_with_split_and_bollinger(ticker, abnormal_returns_df, prices_df, window=20):
    """
    Plot the abnormal returns for a given ticker over time with enhanced visualization,
    and annotate and mark the stock split date. Adds Bollinger Bands to show variance.
    
    Parameters:
    ticker (str): The ticker symbol of the firm whose abnormal returns to plot.
    abnormal_returns_df (pd.DataFrame): The DataFrame containing abnormal returns.
    prices_df (pd.DataFrame): The DataFrame containing stock split dates.
    window (int): The moving average window for calculating Bollinger Bands (default: 20).
    """
    # Check if the ticker exists in the DataFrame
    if ticker not in abnormal_returns_df.columns:
        print(f"Ticker {ticker} not found in the abnormal returns data.")
        return

    # Extract the abnormal returns for the given ticker
    abnormal_data = abnormal_returns_df[ticker].dropna()

    # Ensure 'date' in prices_df is datetime and fetch the split date for the given ticker
    if 'date' in prices_df.columns:
        prices_df['date'] = pd.to_datetime(prices_df['date'], errors='coerce')

    # Extract the split date for the given ticker from the prices DataFrame
    split_date = prices_df.loc[prices_df['Code'] == ticker, 'date'].values
    if len(split_date) > 0:
        split_date = pd.to_datetime(split_date[0])
        print(f"Split date for {ticker}: {split_date}")  # Debugging: print split date
    else:
        print(f"No split date found for ticker {ticker}.")
        split_date = None

    # Ensure that the split date exists in the abnormal data index
    if split_date not in abnormal_data.index:
        print(f"Split date {split_date} not found in abnormal returns data.")
        split_date = None

    # Calculate Bollinger Bands
    rolling_mean = abnormal_data.rolling(window=window).mean()
    rolling_std = abnormal_data.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * 2)  # Two standard deviations above
    lower_band = rolling_mean - (rolling_std * 2)  # Two standard deviations below

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 6))

    # Plot the abnormal returns
    ax.plot(abnormal_data.index, abnormal_data.values, label=f'Abnormal Returns - {ticker}', color='b', lw=2, marker='o')

    # Plot the Bollinger Bands
    ax.plot(abnormal_data.index, upper_band, label='Upper Band (2 Std)', color='grey', linestyle='--', lw=1.5)
    ax.plot(abnormal_data.index, lower_band, label='Lower Band (2 Std)', color='grey', linestyle='--', lw=1.5)
    ax.fill_between(abnormal_data.index, lower_band, upper_band, color='lightgrey', alpha=0.4)

    # Set custom x-axis ticks every 40 points to avoid crowding
    ax.set_xticks(abnormal_data.index[::40])

    # Format the x-axis ticks to display dates (if available) or sequential numbers
    if isinstance(abnormal_data.index, pd.DatetimeIndex):
        ax.set_xticklabels(abnormal_data.index[::40].strftime('%b %d, %Y'), rotation=45, ha='right')
    else:
        ax.set_xticklabels(abnormal_data.index[::40], rotation=45, ha='right')

    # Highlight max and min abnormal returns
    max_return = abnormal_data.max()
    min_return = abnormal_data.min()
    max_date = abnormal_data.idxmax()
    min_date = abnormal_data.idxmin()

    # Annotate the max point
    ax.annotate(f'Max: {max_return:.2f}', xy=(max_date, max_return), xytext=(max_date, max_return + 0.5),
                arrowprops=dict(facecolor='green', shrink=0.05), fontsize=10, color='green')

    # Annotate the min point
    ax.annotate(f'Min: {min_return:.2f}', xy=(min_date, min_return), xytext=(min_date, min_return - 0.5),
                arrowprops=dict(facecolor='red', shrink=0.05), fontsize=10, color='red')

    # Annotate and mark the split date with an arrow and a distinct marker
    if split_date:
        split_return = abnormal_data.loc[split_date]

        # Annotate the split date
        ax.annotate('Split Date', xy=(split_date, split_return), xytext=(split_date, split_return + 1),
                    arrowprops=dict(facecolor='orange', shrink=0.05), fontsize=10, color='orange')

        # Add a marker at the exact split point
        ax.plot(split_date, split_return, marker='x', markersize=12, color='orange', label='Split Date Marker')

    # Add labels and title
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Abnormal Return', fontsize=12)
    ax.set_title(f'Abnormal Returns Over Time for {ticker} (Split Date Marked)', fontsize=14)

    # Add grid, legend, and improve layout
    ax.grid(True)
    ax.legend()
    plt.tight_layout()

    # Display the plot
    plt.show()

# Example usage:
# Assuming `abnormal_returns` is your DataFrame containing abnormal returns,
# and `prices` is your DataFrame containing the stock split dates for each ticker:
plot_abnormal_returns_with_split_and_bollinger("SKIL", abnormal_returns, prices)



Ticker SKIL not found in the abnormal returns data.
